In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fname = "kay_images.npz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/ymnjv/download

In [3]:
with np.load(fname) as dobj:
    dat = dict(**dobj)

In [6]:
# @title rsquare: Compute coefficient of determination of data fit model and RMSE
def rsquare(y,f,varargin=True):
    # % Compute coefficient of determination of data fit model and RMSE
    # %
    # % [r2 rmse] = rsquare(y,f)
    # % [r2 rmse] = rsquare(y,f,c)
    # %
    # % RSQUARE computes the coefficient of determination (R-square) value from
    # % actual data Y and model data F. The code uses a general version of 
    # % R-square, based on comparing the variability of the estimation errors 
    # % with the variability of the original values. RSQUARE also outputs the
    # % root mean squared error (RMSE) for the user's convenience.
    # %
    # % Note: RSQUARE ignores comparisons involving NaN values.
    # % 
    # % INPUTS
    # %   Y       : Actual data
    # %   F       : Model fit
    # %
    # % OPTION
    # %   C       : Constant term in model
    # %             R-square may be a questionable measure of fit when no
    # %             constant term is included in the model.
    # %   [DEFAULT] TRUE : Use traditional R-square computation
    # %            FALSE : Uses alternate R-square computation for model
    # %                    without constant term [R2 = 1 - NORM(Y-F)/NORM(Y)]
    # %
    # % OUTPUT 
    # %   R2      : Coefficient of determination
    # %   RMSE    : Root mean squared error
    # %
    # % EXAMPLE
    # %   x = 0:0.1:10;
    # %   y = 2.*x + 1 + randn(size(x));
    # %   p = polyfit(x,y,1);
    # %   f = polyval(p,x);
    # %   [r2 rmse] = rsquare(y,f);
    # %   figure; plot(x,y,'b-');
    # %   hold on; plot(x,f,'r-');
    # %   title(strcat(['R2 = ' num2str(r2) '; RMSE = ' num2str(rmse)]))
    # %   
    # % Jered R Wells
    # % 11/17/11
    # % jered [dot] wells [at] duke [dot] edu
    # %
    # % v1.2 (02/14/2012)
    # %
    # % Thanks to John D'Errico for useful comments and insight which has helped
    # % to improve this code. His code POLYFITN was consulted in the inclusion of
    # % the C-option (REF. File ID: #34765).

    # By setting varargin as True by default, I think this loop is unnescessary
    # if isempty(varargin); 
    #     c = True; 
    # elif length(varargin)>1; 
    #     error 'Too many input arguments';
    # elif ~islogical(varargin{1}); 
    #     error 'C must be logical (TRUE||FALSE)'
    # else: 
    #     c = varargin{1}; 


    # % Compare inputs
    if (np.size(y) != np.size(f)):      #~all(size(y)==size(f)); 
        sys.exit('Y and F must be the same size')

    # % Check for NaN
    tmp = not (np.isnan(y) or np.isnan(f))   # ~or(isnan(y),isnan(f));
    y = y(tmp)
    f = f(tmp)

    if c: 
        r2 = max(0, ( (1 - np.sum((y[:]-f[:])**2) ) / (np.sum( (y[:]- np.mean(y[:])**2) ) ) ))
    else: 
        r2 = 1 - np.sum((y[:]-f[:])**2) / np.sum((y[:])**2) 
        if r2 < 0:
        # % http://web.maths.unsw.edu.au/~adelle/Garvan/Assays/GoodnessOfFit.html
            #warning('Consider adding a constant term to your model') #ok<WNTAG>
            r2 = 0


    rmse = np.sqrt( np.mean( (y[:] - f[:])**2 ) )

    return r2,rmse

In [7]:
# @title travoxmdl: I think Danny has an improved version of this...? Also seems to have some redundancy w/other programs (not sure which)
def travoxmdl(proj,dataTrnS1,i):
    # % proj--->x
    # % dataTrnS1(i,1:1400)----->Y
    # % dataTrnS1(i,1401:1750)---->Yst
    # % i---->num of the target voxel
    # % for a single voxels
    p = 1400
    q=size(proj,2);
    Y = dataTrnS1[i,1:1400+1]
    Ycnannum = []

    for k in range(1,1400+1): #=1:1400
        if not np.isnan(Y(k)):
            Ycnannum = [Ycnannum,k]

    Y = Y(Ycnannum)
    X = proj
    Xst = X[1401:1750+1,:]
    Xst = [np.ones(350,1),Xst]
    X = X[1:p,:]
    h = np.zeros(q+1,1) 
    Ys = Y - np.mean(Y)
    Ys = Ys.T
    Xp = np.zeros(1400,q)

    for j in range(1,q+1): #=1:q
        Xp[:,j]= ( X[:,j] - np.mean(X[:,j]) ) / np.stdev(X[:,j])

    Xp[np.isnan(Xp)] = 0
    Xp =[np.ones(1400,1),Xp]
    Xp = Xp[Ycnannum,:]   
    h = np.zeros(q+1,1)
    g = np.zeros(q+1,1)
    devi = 100
    deviaf = 99
    devires = 100
    deviresaf = 99
    Yst = dataTrnS1[i,1401:1750+1]
    count = 0
    Ystcnannum = []

    for k in range(1,350+1): #=1:350
        if not np.isnan(Yst(k)):
            Ystcnannum = [Ystcnannum,k]

    Yst = Yst(Ystcnannum)
    Xst = Xst[Ystcnannum,:]

    while (abs(deviaf-devi)>1e-4 and abs(deviresaf-devires)>1e-4):
        devi = deviaf
        devires = deviresaf
        # %h=h-0.001*g;
        g = (Xp @ (Xp*h-Ys)) / norm(Xp @ (Xp*h-Ys) ) + 0.9*g 
            #%eraly stopping grandint descent
        # %g=(Xp'*(Xp*h-Ys));
        h -= 0.001*g
        Ystp = Xst*h
        Yrep = Xp*h
        deviaf = np.sqrt( (Yst.T-Ystp) @ (Yst.T-Ystp) )
        deviresaf = np.sqrt( (Ys-Yrep) @ (Ys-Yrep) )
        count += 1
        fprintf('num:%d devi in stop:%.4f devi set:%.4f\n',count,deviaf,deviresaf);

    h[2:end] = h[2:end] / np.stdev(X).T
    h[isnan(h)] = 0


    return h

In [8]:
# @title linerreg: Needs to be updated...? liner regression function w/ early stopping gradient
def linerreg( X,Y,Xst,Yst ):
# %liner regression function with the early stopping gradint descent
# %   Y=Xh...Xst,Yst is the stopping set
    count = 0
    p = np.size(X,0)      #size(X,1); # p here never shows up...????
    q = np.size(X,1)      #size(X,2);
    h = np.zeros(q,1)  # should this be (p,1)?
    g = np.zeros(q,1)
    RMSE = 100
    RMSEaf = 99
    RMSEres = 100
    RMSEresaf = 99
    R2 = 0
    R2st = 0
    while (RMSEaf<RMSE and RMSEresaf<RMSEres):
        RMSE = RMSEaf
        RMSEres = RMSEresaf
        # %h=h-0.001*g;
        g = (X@(X*h-Y)) / np.linalg.norm(X@(X*h-Y)) + 0.9*g #%eraly stopping grandint descent
        # %g=(X'*(X*h-Y));
        h = h - 0.001 * g
        Ystp = Xst * h
        Yrep = X * h
        [R2,RMSEaf]=rsquare(Yst,Ystp)
        [R2st,RMSEresaf]=rsquare(Y,Yrep)
        count += 1
        if count > 1500:
            break

    # %fprintf('num:%d devist:%.4f devi :%.4f R2:%.4f R2st:%.4f\n',count,RMSEaf,RMSEresaf,R2,R2st);%display the num&devi

    return [ h,RMSEaf,RMSEresaf,R2,R2st,count ]

In [47]:
# @title inittrain: calls linerreg, goes voxel-by-voxel. Does the training...?
def inittrain(proj,dataTrnS1,i):
    # %UNTITLED Summary of this function goes here
    # % proj--->x
    # % dataTrnS1(i,1:1400)----->Y
    # % dataTrnS1(i,1401:1750)---->Yst
    # % i---->num of the target voxel
    # % for a single voxels
    p = 1550
    q = np.size(proj,1) #size(proj,2);
    Y = dataTrnS1[i,1:p]
    Ycnannum = []

    #for k in range(0,p):
    #    if not np.isnan(Y[k]): #~isnan(Y(k))
    #        Ycnannum = [Ycnannum,k] #% figure out the nan element

    #Y = Y(Ycnannum) #%exclude the nan element
    X = proj 
    Xst = X[(p+1):(np.size(proj,0)+1),:]
    Xst = [np.ones((np.size(proj,1)-p,1)),Xst]
    X = X[1:p,:]
    h = np.zeros((q+1,1))
    Ys = Y - np.mean(Y)
    Ys = Ys.T
    Xp = np.zeros((p,q))

    for j in range (1,q+1): #=1:q
        Xp[:,j] = (X[:,j] - np.mean(X[:,j]) ) / np.std(X[:,j])

    Xp[np.isnan(Xp)] = 0
    Xp = [np.ones(p,1),Xp]
    Xp = Xp[Ycnannum,:]   
    Yst = dataTrnS1[i, (p+1):size(proj,1)]
    Ystcnannum = []

    for k in range(1,np.size(proj,0)-p+1):  #=1:size(proj,1)-p
        if not np.isnan(Yst(k)):
            Ystcnannum = [Ystcnannum,k]

    Yst = Yst(Ystcnannum).T
    Xst = Xst[Ystcnannum,:]
    h,RMSEaf,RMSEresaf,R2,R2st,count = linerreg(Xp,Ys,Xst,Yst) #% liner regression function
    # %fprintf('num:%d devist:%.4f devi :%.4f R2:%.4f R2st:%.4f\n',count,RMSEaf,RMSEresaf,R2,R2st);
    h[2:end] /= np.std(X)                 # h(2:end)=h(2:end)./std(X)';
    h[np.isnan(h)] = 0

    return [ h,RMSEaf,RMSEresaf,R2,R2st,count ]

In [80]:
X_small = np.load("proj_small.npy")
X_test = X_small[4]
X_small = X_small[:4]

In [81]:
Y_small = dat["responses"][:4]
Y_test = dat["responses"][4]

In [82]:
print("X shape: ", X_small.shape)
print("Y shape: ", Y_small.shape)

print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

X shape:  (4, 2728)
Y shape:  (4, 8428)
X_test shape:  (2728,)
Y_test shape:  (8428,)


In [97]:
print(Y_small[:,0].shape)

coef_init = np.zeros(X_small.shape[1])
print(coef_init)

(4,)
[0. 0. 0. ... 0. 0. 0.]


In [104]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(max_iter=100, tol=1e-3)
reg.fit(X_small, Y_small[:,0], coef_init = coef_init)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=100,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [105]:
thetas = reg.coef_
print(thetas.shape)
print(thetas[0])

(2728,)
15979047042.004503


In [106]:
Y_hat = X_test.T @ thetas
print(Y_hat)

-2096123034400.916


In [107]:
print(Y_test[0])

0.8427713325267792
